## Exercise 1


### Preprocessing


In [48]:
# General imports
import pandas as pd
import sklearn as sk
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import scipy as sc

sns.set_theme()

In [49]:
# fetch dataset
housing = pd.read_csv("./datasets/housing/housing.csv")

In [50]:
# Create a test set
from sklearn.model_selection import train_test_split

X = housing.drop("median_house_value", axis=1)
y = housing["median_house_value"]

X["income_category"] = pd.cut(
    housing["median_income"],
    bins=[0, 1.5, 3, 4.5, 6, np.inf],
    labels=["lt_15k", "15_to_30k", "30_to_45k", "45_to_60k", "mt_60k"],
)

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=X["income_category"]
)

for df in [X_train, X_test]:
    df.drop("income_category", axis=1, inplace=True)

In [51]:
# create the class ClusterSimilarity, which gets the 10 most important population centers and computes the similarity of each row to each center
# based on the latitude and longitude
from sklearn.cluster import KMeans
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.metrics.pairwise import rbf_kernel


class ClusterSimilarity(BaseEstimator, TransformerMixin):
    def __init__(self, n_clusters=10, gamma=1.0, random_state=None):
        self.n_clusters = n_clusters
        self.gamma = gamma
        self.random_state = random_state

    def fit(self, X, y=None, sample_weight=None):
        self.kmeans_ = KMeans(self.n_clusters, random_state=self.random_state)
        self.kmeans_.fit(X, sample_weight=sample_weight)
        return self  # always return self!

    def transform(self, X):
        return rbf_kernel(X, self.kmeans_.cluster_centers_, gamma=self.gamma)

    def get_feature_names_out(self, names=None):
        return [f"Cluster {i} similarity" for i in range(self.n_clusters)]

In [52]:
# Preprocessing
from sklearn.preprocessing import StandardScaler, FunctionTransformer, OneHotEncoder
from sklearn.impute import SimpleImputer
from sklearn.pipeline import make_pipeline, Pipeline
from sklearn.compose import ColumnTransformer, make_column_selector


def column_ratio(X):
    return X[:, [0]] / X[:, [1]]


def ratio_name(function_transformer, feature_names_in):
    return ["ratio"]  # feature names out


def ratio_pipeline():
    return make_pipeline(
        SimpleImputer(strategy="median"),
        FunctionTransformer(column_ratio, feature_names_out=ratio_name),
        StandardScaler(),
    )


cat_pipeline = Pipeline(
    [
        ("most_freq_imputer", SimpleImputer(strategy="most_frequent")),
        ("one_hot_encoder", OneHotEncoder(handle_unknown="ignore")),
    ]
)

log_pipeline = make_pipeline(
    SimpleImputer(strategy="median"),
    FunctionTransformer(np.log, feature_names_out="one-to-one"),
    StandardScaler(),
)

cluster_simil = ClusterSimilarity(n_clusters=10, gamma=1.0, random_state=42)

default_num_pipeline = make_pipeline(SimpleImputer(strategy="median"), StandardScaler())

preprocessing = ColumnTransformer(
    [
        ("bedrooms", ratio_pipeline(), ["total_bedrooms", "total_rooms"]),
        ("rooms_per_house", ratio_pipeline(), ["total_rooms", "households"]),
        ("people_per_house", ratio_pipeline(), ["population", "households"]),
        (
            "log",
            log_pipeline,
            [
                "total_bedrooms",
                "total_rooms",
                "population",
                "households",
                "median_income",
            ],
        ),
        ("geo", cluster_simil, ["latitude", "longitude"]),
        ("cat", cat_pipeline, make_column_selector(dtype_include=object)),
    ],
    remainder=default_num_pipeline,
)  # one column remaining: housing_median_age

### Model


In [53]:
# Only use the first 5_000 rows of the train set
X_train_small = X_train[:5000].copy()
y_train_small = y_train[:5000].copy()

In [54]:
from sklearn.svm import SVR

svr_model = Pipeline([("preprocessing", preprocessing), ("svr", SVR())])

### First Grid Search


In [55]:
import warnings

warnings.filterwarnings("ignore")

In [56]:
from sklearn.model_selection import GridSearchCV

param_grid = [
    {
        "svr__kernel": ["linear"],
        "svr__C": [10.0, 30.0, 100.0, 300.0, 1000.0, 3000.0, 10000.0, 30000.0],
    },
    {
        "svr__kernel": ["rbf"],
        "svr__C": [1.0, 3.0, 10.0, 30.0, 100.0, 300.0, 1000.0],
        "svr__gamma": [0.01, 0.03, 0.1, 0.3, 1.0, 3.0],
    },
]
grid_search = GridSearchCV(
    svr_model,
    param_grid=param_grid,
    cv=3,
    scoring="neg_root_mean_squared_error",
)

grid_search.fit(X_train_small, y_train_small)

GridSearchCV(cv=3,
             estimator=Pipeline(steps=[('preprocessing',
                                        ColumnTransformer(remainder=Pipeline(steps=[('simpleimputer',
                                                                                     SimpleImputer(strategy='median')),
                                                                                    ('standardscaler',
                                                                                     StandardScaler())]),
                                                          transformers=[('bedrooms',
                                                                         Pipeline(steps=[('simpleimputer',
                                                                                          SimpleImputer(strategy='median')),
                                                                                         ('functiontransformer',
                                                                                          FunctionTransformer(feature_names_out=<f...
                                                                         <sklearn.compose._column_transformer.make_column_selector object at 0x7f80b6fb5850>)])),
                                       ('svr', SVR())]),
             param_grid=[{'svr__C': [10.0, 30.0, 100.0, 300.0, 1000.0, 3000.0,
                                     10000.0, 30000.0],
                          'svr__kernel': ['linear']},
                         {'svr__C': [1.0, 3.0, 10.0, 30.0, 100.0, 300.0,
                                     1000.0],
                          'svr__gamma': [0.01, 0.03, 0.1, 0.3, 1.0, 3.0],
                          'svr__kernel': ['rbf']}],
             scoring='neg_root_mean_squared_error')

In [61]:
svr_grid_search_rmse = -grid_search.best_score_
svr_grid_search_rmse

73383.16676986746

In [62]:
grid_search.best_params_

{'svr__C': 6900.0, 'svr__kernel': 'linear'}

In [57]:
cv_results = pd.DataFrame(grid_search.cv_results_)
cv_results.sort_values(by="rank_test_score").head()

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_svr__C,param_svr__kernel,param_svr__gamma,params,split0_test_score,split1_test_score,split2_test_score,mean_test_score,std_test_score,rank_test_score
5,0.384856,0.011152,0.108682,0.005004,3000.0,linear,NaN,"{'svr__C': 3000.0, 'svr__kernel': 'linear'}",-73861.232727,-69255.894863,-76635.326142,-73250.817911,3043.403378,1
6,0.470989,0.034529,0.107654,0.003984,10000.0,linear,NaN,"{'svr__C': 10000.0, 'svr__kernel': 'linear'}",-73924.598897,-69069.445802,-77540.469530,-73511.504743,3470.595130,2
4,0.403137,0.064835,0.104943,0.003678,1000.0,linear,NaN,"{'svr__C': 1000.0, 'svr__kernel': 'linear'}",-74275.078618,-70017.497461,-77467.196331,-73919.924137,3051.677609,3
7,0.781546,0.154519,0.106374,0.000475,30000.0,linear,NaN,"{'svr__C': 30000.0, 'svr__kernel': 'linear'}",-73784.561792,-69297.350275,-80208.042665,-74429.984911,4477.590897,4
3,0.366323,0.030128,0.104377,0.002632,300.0,linear,NaN,"{'svr__C': 300.0, 'svr__kernel': 'linear'}",-75845.875453,-72221.517023,-79801.434914,-75956.275797,3095.473037,5


The lowest error is achieved with the highest value of C. We should try higher values of C until we find a maximum.


### Second Grid Search


In [58]:
from sklearn.model_selection import GridSearchCV

param_grid = [
    {"svr__kernel": ["linear"], "svr__C": np.linspace(6_900, 7_100, 11)},
]

grid_search = GridSearchCV(
    svr_model,
    param_grid=param_grid,
    cv=6,
    scoring="neg_root_mean_squared_error",
    verbose=0,
)

grid_search.fit(X_train_small, y_train_small)

GridSearchCV(cv=6,
             estimator=Pipeline(steps=[('preprocessing',
                                        ColumnTransformer(remainder=Pipeline(steps=[('simpleimputer',
                                                                                     SimpleImputer(strategy='median')),
                                                                                    ('standardscaler',
                                                                                     StandardScaler())]),
                                                          transformers=[('bedrooms',
                                                                         Pipeline(steps=[('simpleimputer',
                                                                                          SimpleImputer(strategy='median')),
                                                                                         ('functiontransformer',
                                                                                          FunctionTransformer(feature_names_out=<f...
                                                                                          SimpleImputer(strategy='most_frequent')),
                                                                                         ('one_hot_encoder',
                                                                                          OneHotEncoder(handle_unknown='ignore'))]),
                                                                         <sklearn.compose._column_transformer.make_column_selector object at 0x7f80b6fb5850>)])),
                                       ('svr', SVR())]),
             param_grid=[{'svr__C': array([6900., 6920., 6940., 6960., 6980., 7000., 7020., 7040., 7060.,
       7080., 7100.]),
                          'svr__kernel': ['linear']}],
             scoring='neg_root_mean_squared_error')

In [59]:
cv_results = pd.DataFrame(grid_search.cv_results_)
cv_results.sort_values(by="rank_test_score").head()

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_svr__C,param_svr__kernel,params,split0_test_score,split1_test_score,split2_test_score,split3_test_score,split4_test_score,split5_test_score,mean_test_score,std_test_score,rank_test_score
0,0.705650,0.043206,0.068426,0.002251,6900.0,linear,"{'svr__C': 6900.0, 'svr__kernel': 'linear'}",-79853.874529,-66700.446555,-65324.970590,-72623.521359,-75617.019597,-80179.167989,-73383.166770,5819.738342,1
2,0.719677,0.054073,0.070086,0.001514,6940.0,linear,"{'svr__C': 6940.0, 'svr__kernel': 'linear'}",-79853.861595,-66697.315773,-65317.803643,-72618.059829,-75615.313583,-80197.895214,-73383.374939,5825.646874,2
1,0.770540,0.082470,0.068605,0.003875,6920.0,linear,"{'svr__C': 6920.0, 'svr__kernel': 'linear'}",-79853.861478,-66699.131710,-65323.568004,-72623.416114,-75615.664594,-80188.524479,-73384.027730,5822.048661,3
3,0.729225,0.050083,0.069551,0.002303,6960.0,linear,"{'svr__C': 6960.0, 'svr__kernel': 'linear'}",-79854.169851,-66695.598881,-65317.661596,-72616.329029,-75615.347933,-80207.344783,-73384.408679,5827.948295,4
4,0.697188,0.017906,0.073327,0.008925,6980.0,linear,"{'svr__C': 6980.0, 'svr__kernel': 'linear'}",-79854.170022,-66694.581778,-65318.239487,-72619.908827,-75615.817570,-80212.311680,-73385.838227,5828.930348,5


### Conclusion


At first, it appeared that the error would continue to go down as C increased to infinity. This would be a problem because increasing C decreases regularization and makes the model more prone to overfitting.

After a more exhaustive grid search it was clear that this is not true, the optimal value of C is 6940. Still, knowing the default value is C=1, it is not clear if such a high value will be prone to overfitting.

The best model achieved an RMSE of 73_700, which is worse than the best random forest model (RMSE=44_000)


## Exercise 2


In [60]:
from sklearn.model_selection import RandomizedSearchCV
from scipy.stats import loguniform, expon

param_distributions = [
    {
        "svr__kernel": ["linear"],
        "svr__C": loguniform(20, 200_000),
    },
    {
        "svr__kernel": ["rbf"],
        "svr__C": loguniform(20, 200_000),
        "svr__gamma": expon(scale=1),
    },
]

rng_search = RandomizedSearchCV(
    svr_model,
    param_distributions=param_distributions,
    n_iter=50,
    cv=3,
    scoring="neg_root_mean_squared_error",
    verbose=2,
    random_state=42,
)

rng_search.fit(X_train_small, y_train_small)

Fitting 3 folds for each of 50 candidates, totalling 150 fits
[CV] END ......svr__C=30704.493883946954, svr__kernel=linear; total time=   1.3s
[CV] END ......svr__C=30704.493883946954, svr__kernel=linear; total time=   0.8s
[CV] END ......svr__C=30704.493883946954, svr__kernel=linear; total time=   0.8s
[CV] END ......svr__C=16943.602837639955, svr__kernel=linear; total time=   0.7s
[CV] END ......svr__C=16943.602837639955, svr__kernel=linear; total time=   0.6s
[CV] END ......svr__C=16943.602837639955, svr__kernel=linear; total time=   0.7s
[CV] END ........svr__C=4880.12141816351, svr__kernel=linear; total time=   0.5s
[CV] END ........svr__C=4880.12141816351, svr__kernel=linear; total time=   0.5s
[CV] END ........svr__C=4880.12141816351, svr__kernel=linear; total time=   0.5s
[CV] END svr__C=84.1410790057587, svr__gamma=0.059838768608680676, svr__kernel=rbf; total time=   0.7s
[CV] END svr__C=84.1410790057587, svr__gamma=0.059838768608680676, svr__kernel=rbf; total time=   0.7s
[CV

[CV] END svr__C=432.3788481314884, svr__gamma=0.15416196746656105, svr__kernel=rbf; total time=   0.8s
[CV] END svr__C=432.3788481314884, svr__gamma=0.15416196746656105, svr__kernel=rbf; total time=   0.9s
[CV] END ......svr__C=24.175082946113907, svr__kernel=linear; total time=   0.5s
[CV] END ......svr__C=24.175082946113907, svr__kernel=linear; total time=   0.6s
[CV] END ......svr__C=24.175082946113907, svr__kernel=linear; total time=   0.5s
[CV] END svr__C=15453.436955926874, svr__gamma=2.789575528819812, svr__kernel=rbf; total time=   0.8s
[CV] END svr__C=15453.436955926874, svr__gamma=2.789575528819812, svr__kernel=rbf; total time=   0.8s
[CV] END svr__C=15453.436955926874, svr__gamma=2.789575528819812, svr__kernel=rbf; total time=   0.8s
[CV] END svr__C=106.74065525207914, svr__gamma=0.20261142283225705, svr__kernel=rbf; total time=   0.8s
[CV] END svr__C=106.74065525207914, svr__gamma=0.20261142283225705, svr__kernel=rbf; total time=   0.7s
[CV] END svr__C=106.74065525207914, s

KeyboardInterrupt: 

In [ ]:
cv_results = pd.DataFrame(rng_search.cv_results_)
cv_results.sort_values(by="rank_test_score").head()

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_svr__C,param_svr__kernel,param_svr__gamma,params,split0_test_score,split1_test_score,split2_test_score,mean_test_score,std_test_score,rank_test_score
96,1.976864,0.200507,0.271421,0.028022,144207.904734,rbf,0.80798,"{'svr__C': 144207.90473431294, 'svr__gamma': 0...",-69228.479576,-67063.866180,-66194.374891,-67495.573549,1275.728877,1
65,1.690086,0.027878,0.245484,0.003251,133464.122239,rbf,1.032797,"{'svr__C': 133464.12223913567, 'svr__gamma': 1...",-69334.631375,-66988.428714,-66501.073188,-67608.044426,1236.987125,2
15,1.716324,0.022108,0.253119,0.000789,120144.989518,rbf,0.737827,"{'svr__C': 120144.98951812398, 'svr__gamma': 0...",-69573.091896,-67055.979724,-66299.673462,-67642.915027,1399.330042,3
21,1.771684,0.053812,0.259181,0.001789,119130.320374,rbf,1.145139,"{'svr__C': 119130.32037421125, 'svr__gamma': 1...",-69515.868575,-66989.808041,-66576.583992,-67694.086869,1299.193331,4
80,1.729660,0.079671,0.271645,0.022117,57645.203686,rbf,0.530003,"{'svr__C': 57645.203685565466, 'svr__gamma': 0...",-70953.146660,-67299.180035,-66537.892155,-68263.406283,1927.159789,5


In [ ]:
from sklearn.model_selection import cross_val_score

svr_model = rng_search.best_estimator_
score = -cross_val_score(
    svr_model, X_train_small, y_train_small, scoring="neg_root_mean_squared_error", cv=5
)

The randomized search achieved a much better result: RMSE=56_809. It found hyperparameters that make the rbf kernel work.

However, this wasn't achieved on the first try, it was achieved after looking up the solution, which hinted that one should use `expon(scale=1)` for the gamma distribution. It is not clear to me how could you know this beforehand.


## Exercise 3


In [ ]:
from sklearn.feature_selection import SelectFromModel
from sklearn.ensemble import RandomForestRegressor

# We'll use a random forest as the estimator from which the SelectFromModel transformer is build
random_forest = RandomForestRegressor(max_features=6, random_state=42)
selector = SelectFromModel(estimator=random_forest, max_features=20)

svr = svr_model

svr_selector_model = Pipeline(
    [("preprocessing", preprocessing), ("feature_selector", selector), ("svr", svr)]
)

Next, we'll do a grid search of the feature_selector\_\_max_features hyperparameter to see how many features should the selector keep.


In [ ]:
param_grid = [
    {"feature_selector__max_features": np.linspace(4, 24, 21).astype("int")},
]

grid_search = GridSearchCV(
    svr_selector_model,
    param_grid=param_grid,
    cv=3,
    scoring="neg_root_mean_squared_error",
    verbose=0,
)

grid_search.fit(X_train_small, y_train_small)

GridSearchCV(cv=3,
             estimator=Pipeline(steps=[('preprocessing',
                                        ColumnTransformer(remainder=Pipeline(steps=[('simpleimputer',
                                                                                     SimpleImputer(strategy='median')),
                                                                                    ('standardscaler',
                                                                                     StandardScaler())]),
                                                          transformers=[('bedrooms',
                                                                         Pipeline(steps=[('simpleimputer',
                                                                                          SimpleImputer(strategy='median')),
                                                                                         ('functiontransformer',
                                                                                          FunctionTransformer(feature_names_out=<f...
                                                                         <sklearn.compose._column_transformer.make_column_selector object at 0x7f80c8423190>)])),
                                       ('feature_selector',
                                        SelectFromModel(estimator=RandomForestRegressor(max_features=6,
                                                                                        random_state=0),
                                                        max_features=20)),
                                       ('svr', SVR(C=134787, gamma=0.151231))]),
             param_grid=[{'feature_selector__max_features': array([ 4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20,
       21, 22, 23, 24])}],
             scoring='neg_root_mean_squared_error')

In [ ]:
cv_results = pd.DataFrame(grid_search.cv_results_)
cv_results.sort_values(by="rank_test_score", ascending=True).head()

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_feature_selector__max_features,params,split0_test_score,split1_test_score,split2_test_score,mean_test_score,std_test_score,rank_test_score
10,1.803566,0.031350,0.296035,0.033230,14,{'feature_selector__max_features': 14},-71406.481704,-67575.410405,-67454.872542,-68812.254884,1835.055302,1
16,1.809562,0.054932,0.291838,0.023069,20,{'feature_selector__max_features': 20},-71406.481704,-67575.410405,-67454.872542,-68812.254884,1835.055302,1
15,1.857478,0.079563,0.278640,0.010570,19,{'feature_selector__max_features': 19},-71406.481704,-67575.410405,-67454.872542,-68812.254884,1835.055302,1
14,1.890936,0.044892,0.267239,0.009159,18,{'feature_selector__max_features': 18},-71406.481704,-67575.410405,-67454.872542,-68812.254884,1835.055302,1
13,1.842864,0.128858,0.279102,0.009253,17,{'feature_selector__max_features': 17},-71406.481704,-67575.410405,-67454.872542,-68812.254884,1835.055302,1


In [ ]:
cv_results.sort_values(
    by="param_feature_selector__max_features", ascending=False
).head()

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_feature_selector__max_features,params,split0_test_score,split1_test_score,split2_test_score,mean_test_score,std_test_score,rank_test_score
20,1.186799,0.791756,0.193189,0.138801,24,{'feature_selector__max_features': 24},-71406.481704,NaN,-67454.872542,NaN,NaN,21
19,1.769080,0.072194,0.284580,0.042895,23,{'feature_selector__max_features': 23},-71406.481704,-67575.410405,-67454.872542,-68812.254884,1835.055302,1
18,1.831855,0.046891,0.261458,0.002815,22,{'feature_selector__max_features': 22},-71406.481704,-67575.410405,-67454.872542,-68812.254884,1835.055302,1
17,1.788361,0.052294,0.263138,0.007462,21,{'feature_selector__max_features': 21},-71406.481704,-67575.410405,-67454.872542,-68812.254884,1835.055302,1
16,1.809562,0.054932,0.291838,0.023069,20,{'feature_selector__max_features': 20},-71406.481704,-67575.410405,-67454.872542,-68812.254884,1835.055302,1


I'm having a lot of trouble replicating the same results as Geron gets in the solutions.
